In [1]:
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dot, Flatten
from sklearn.model_selection import train_test_split

In [2]:
projects = pd.read_csv('new_data/project_data.csv')
ratings = pd.read_csv('new_data/generated_data.csv')

In [30]:
n_project = ratings.id_project.nunique()
n_freelancer = ratings.id_freelancer.nunique()

In [4]:
train, test = train_test_split(ratings, test_size=0.2)

In [5]:
EMBEDDING_DIM = 36

#input layer
project_input = Input(shape=1)
freelancer_input = Input(shape=1)

#embedding layer
project_embedding = Embedding(n_project+1, EMBEDDING_DIM)(project_input)
freelancer_embedding = Embedding(n_freelancer+1, EMBEDDING_DIM)(freelancer_input)

#flatten layer
project_flat = Flatten()(project_embedding)
freelancer_flat = Flatten()(freelancer_embedding)

#output layer
output = Dot(1)([project_flat, freelancer_flat])

model = Model([project_input, freelancer_input],[output])

In [6]:
model.compile(optimizer='adam', loss='mse')

In [7]:
model.fit(x=[train.id_project, train.id_freelancer], y=[train.rating_for_freelancer],
         epochs=64, batch_size=128)

Epoch 1/64
3/3 [==============================] - 1s 11ms/step - loss: 4.8160
Epoch 2/64
3/3 [==============================] - 0s 7ms/step - loss: 4.8115
Epoch 3/64
3/3 [==============================] - 0s 8ms/step - loss: 4.8075
Epoch 4/64
3/3 [==============================] - 0s 7ms/step - loss: 4.8032
Epoch 5/64
3/3 [==============================] - 0s 5ms/step - loss: 4.7981
Epoch 6/64
3/3 [==============================] - 0s 6ms/step - loss: 4.7922
Epoch 7/64
3/3 [==============================] - 0s 7ms/step - loss: 4.7852
Epoch 8/64
3/3 [==============================] - 0s 5ms/step - loss: 4.7767
Epoch 9/64
3/3 [==============================] - 0s 6ms/step - loss: 4.7667
Epoch 10/64
3/3 [==============================] - 0s 7ms/step - loss: 4.7544
Epoch 11/64
3/3 [==============================] - 0s 6ms/step - loss: 4.7401
Epoch 12/64
3/3 [==============================] - 0s 5ms/step - loss: 4.7230
Epoch 13/64
3/3 [==============================] - 0s 5ms/step - loss: 4

In [8]:
model.evaluate(x=[test.id_project, test.id_freelancer], y=[test.rating_for_freelancer])

3/3 [==============================] - 0s 5ms/step - loss: 0.5063


0.5062746405601501

In [10]:
predictions = model.predict([test.id_project, test.id_freelancer])

3/3 [==============================] - 0s 3ms/step


In [11]:
for i in range(10):
    print(f"Actual Rating: {test.rating_for_freelancer.iloc[i]}, Predicted Rating: {predictions[i][0]}")

Actual Rating: 2, Predicted Rating: 1.6382410526275635
Actual Rating: 2, Predicted Rating: 1.5705326795578003
Actual Rating: 2, Predicted Rating: 1.564210057258606
Actual Rating: 2, Predicted Rating: 1.7386120557785034
Actual Rating: 2, Predicted Rating: 1.5537513494491577
Actual Rating: 2, Predicted Rating: 1.6484813690185547
Actual Rating: 2, Predicted Rating: 1.5809624195098877
Actual Rating: 1, Predicted Rating: 1.5776418447494507
Actual Rating: 2, Predicted Rating: 1.5458073616027832
Actual Rating: 2, Predicted Rating: 1.562220811843872


In [12]:
from sklearn.metrics import mean_squared_error

# Menghitung Mean Squared Error (MSE) pada set data uji
mse = mean_squared_error(test.rating_for_freelancer, predictions)
print(f'Mean Squared Error on Test Data: {mse}')


Mean Squared Error on Test Data: 0.5062746518948945


In [23]:
import numpy as np
import matplotlib.pyplot as plt

def get_recommendations(id_freelancer, projects, model):
    projects = projects.copy()
    id_freelancers = np.array([id_freelancer] * len(projects))
    results = model([projects.id_project.values, id_freelancers]).numpy().reshape(-1)

    projects['predicted_rating'] = pd.Series(results)
    projects = projects.sort_values('predicted_rating', ascending=False)

    print(f'Recommendations for user {id_freelancer}')
    return(projects)

get_recommendations(8, projects, model)

Recommendations for user 8


,id_project,name_project,project_desc,id_customer,due_project,createdAt,updatedAt,category_game,game,predicted_rating
6,7,Raid Assistance,Need help beating a challenging raid or boss i...,456,2023-12-31,2023-12-21,2023-12-21,MMO,Final Fantasy XIV,1.763506
37,38,Top-tier PvP Coaching,Seeking a pro-gamer coach to reach the top tie...,6789012,2025-01-01,2024-01-22,2024-01-22,MOBA,League of Legends,1.762444
3,4,Item Farming,Need help farming items or resources in a game.,72,2024-02-15,2023-12-19,2023-12-19,MMO,World of Warcraft,1.755718
47,47,Mobile Game Strategy Guide,Need a pro gamer to create a comprehensive str...,1234567,2025-07-15,2024-03-25,2024-03-25,Mobile,Mobile Legends: Bang Bang,1.752283
16,17,In-Game Photography,Hiring a photographer to capture stunning in-g...,2345,2024-05-15,2024-01-01,2024-01-01,Simulation,Stardew Valley,1.749135
8,9,Coaching Service,Need a skilled coach to analyze my gameplay an...,1234,2024-01-10,2023-12-23,2023-12-23,FPS,Valorant,1.743647
35,36,Top-Rank Boosting,Need a pro gamer to boost my account to the to...,34567890,2025-03-15,2024-02-15,2024-02-15,FPS,Valorant,1.737977
43,44,Horror Game Live Stream Collaboration,Collaborate with us for a live stream event fe...,444,2025-03-01,2024-03-15,2024-03-15,Horror,Resident Evil Village,1.734319
44,44,Open World Exploration Guide,Seeking a pro gamer to create a comprehensive ...,10987,2025-08-01,2024-04-04,2024-04-04,Open World,The Legend of Zelda: Breath of the Wild,1.734319
1,2,Grinding Help,Need help leveling up my character or account ...,45,2024-01-15,2023-12-19,2023-12-19,RPG,Genshin Impact,1.712336
